# Background Tasks

> Background tasks are functions run after handlers return a response.

Useful for operations where the users gets a response quickly but doesn't need to wait for the operation to finish. Typical scenarios include:

- User setup in complex systems where you can inform the user and other people later in email that their account is complete
- Batch processes that can take a significant amount of time (bulk email or API calls)
- Any other process where the user can be notified later by email, websocket, webhook, or pop-up

Background tasks are an easy-to-use wrapper over Python's async and threading libraries. Used to improve user experience, background tasks often making apps feel faster to the end user.

## A simple background task example

In this example attaching a task to FtResponse by assigning it via the background argument. When the page is visited, it will display 'Simple Background Task Example' almost instantly, while in the terminal it will slowly count upward from 0.

``` {.python filename="main.py" code-line-numbers="true"}
from fasthtml.common import *
from starlette.background import BackgroundTask
from time import time, sleep

app, rt = fast_app()

def counter(loops:int): # <1>
    """Slowly print integers to the terminal"""
    for i in range(loops):
        print(i)
        sleep(i)

@rt
def index():
    task = BackgroundTask(counter, loops=5)  # <2>
    cts = Titled('Simple Background Task Example')
    return FtResponse(cts, background=task) # <3>

serve()
```

1. `counter` is our task function. There is nothing special about it, although it is a good practice for its arguments to be serializable as JSON
2. We use `starlette.background.BackgroundTask` to turn the task_counter function into a background task
3. `FtResponse` is called explicitly so we can attack the task to its background. Normally we don't need to call `FtResponse` explicitly, setting background tasks is the exception.

## A more realistic example

Let's imagine that we are accessing a slow-to-process critical service. We don't want our users to have to wait. While we could set up SSE to notify on completion, instead we decide to periodically check to see if the status of their record has changed.

First, create a very simple slow timestamp API. All it does is stall requests for a few seconds before returning JSON containing timestamps.

```python
# slow_api.py
from fasthtml.common import *
from time import sleep, time

app, rt = fast_app()

@rt('/slow/{ts}')
def slow(ts: int):
    sleep(3)
    return dict(ts=ts, response_ts=int(time()))

serve(port=8123)
```

Now let's create a website that uses this API to fetch the timestamp from the glacially slow service.

```python
# main.py
from fasthtml.common import *
from fastlite import *
from starlette.background import BackgroundTask
import time
import httpx

app, rt = fast_app()

db = database(':memory:')

class Moment: ts: int; response_ts: int
moments = db.create(Moment, pk='ts')

def task_submit(ts:int): # <1>
    # This task function calls the slow service, representing a slow process that
    # we've encapsulated in a function.
    client = httpx.Client()
    r = client.post(f'http://127.0.0.1:8123/slow/{ts}')
    moments.insert(**r.json())

@rt
def submit():
    ts = int(time.time())
    task = BackgroundTask(task_submit, ts=ts) # <2> 
    cts = P(f'Submitted: {ts}')
    return FtResponse(cts, background=task) # <3> 

@rt
def showmoments(): return Ul(*[Li(m) for m in moments()])

@rt
def index():
    return Titled('Background Task Dashboard',
        P(Button('Press to call slow service', # <4> 
            hx_post='/submit', hx_target='#res')),
        P('', id='res'),
        Div(Ul(*[Li(m) for m in moments()]),
            hx_get=showmoments, hx_trigger='every 5s'), # <5>
    )

serve()
```

1. It is common, but not necessary to prefix task functions with 'task_'
2. Create a background task by passing in the function to a BackgroundTask object, followed by any arguments.
3. In FtResponse, use the background keyword argument to set the task to be run after the HTTP Response is generated.
4. When this button is pressed, the 'submit' handler will respond instantly. The task_submit function will insert the slow API response into the db later. 
5. Every 5 seconds get the moments stored in the DB.

::: {.callout-tip}

In the example above we use a synchronous background task function set in the `FtResponse` of a synchronous handler. However, we can also use asynchronous functions and handlers.

:::

## Multiple background tasks in a handler

It is possible to add multiple background tasks to an FtResponse.

```python
from fasthtml.common import *
from starlette.background import BackgroundTasks

@rt
async def signup(email, username):
    tasks = BackgroundTasks()
    tasks.add_task(send_welcome_email, to_address=email)
    tasks.add_task(send_admin_notification, username=username)
    cts = Titled('Signup successful!')
    return FtResponse(cts, background=tasks)

async def send_welcome_email(to_address):
    ...

async def send_admin_notification(username):
    ...
```

::: {.callout-caution}
## Background tasks are not distributed task queues

While background tasks often provide the user with a faster experience, the server itself isn't accelerated. What that means is that processes are still happening, but are hidden. So if a server is struggling under the load of a lot of user activity, so long as it isn't an issue with open HTTP connections consuming resources, background tasks won't help with server load.

This is where full-fledged distributed task queue libraries like Celery and Dramatiq come into play. At the cost of dramatically increased complexity over background tasks, they allow for the distribution of tasks over additional servers. These libraries also provide improved observability, retry mechanisms, and persistence in case of server shutdown.

In our experience, most of the time background tasks like the ones built into FastHTML suffice. We recommend trying out background tasks before using a distributed task queue. If background task functions are written with JSON serializable arguments, then converting the functions to work in a distributed task queue is a straight-forward process.
:::